In [1]:
# TODO: switch back to main branch after iterations
!test -d /content/aquathon-2025 || ( test -d /content && git clone --single-branch --branch colab-iteration --depth=4 https://github.com/neradis/aquathon-2025.git /content/aquathon-2025 )
!test -d /content/aquathon-2025 && git -C /content/aquathon-2025 pull

!test -d /content/aquathon-2025/data-augmentation && pip install -r /content/aquathon-2025/data-augmentation/requirements-google-colab.txt'

In [2]:
from os import path as osp
import sys

if osp.exists('/content/aquathon-2025/data-augmentation'):
    sys.path.insert(0,'/content/aquathon-2025/data-augmentation')

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False


In [3]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

import tsgm
import tensorflow as tf
from tensorflow import keras

import numpy as np

 
import load_data

2025-11-13 12:09:33.289381: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-13 12:09:33.341424: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-13 12:09:35.083382: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.




Failed to import TF-Keras. Please note that TF-Keras is not installed by default when you install TensorFlow Probability. This is so that JAX-only users do not have to install TensorFlow or TF-Keras. To use TensorFlow Probability with TensorFlow, please install the tf-keras or tf-keras-nightly package.
This can be be done through installing the tensorflow-probability[tf] extra.




Failed to import TF-Keras. Please note that TF-Keras is not installed by default when you install TensorFlow Probability. This is so that JAX-only users do not have to install TensorFlow or TF-Keras. To use TensorFlow Probability with TensorFlow, please install the tf-keras or tf-keras-nightly package.
This can be be done through installing the tensorflow-probability[tf] extra.


TensorFlow not installed. STS model is not available.


In [ ]:
production_df = load_data.load_from_db(in_colab=IN_COLAB)

actual_production = tf.convert_to_tensor(production_df[['actual']], dtype=np.float32)
actual_production_tsgm = tf.expand_dims(actual_production, axis=0)

In [ ]:
scaler = tsgm.utils.TSFeatureWiseScaler()
scaled_actual_production = scaler.fit_transform(actual_production_tsgm)

In [ ]:
# input-data-specific hyperparameters
seq_len, n_features = scaled_actual_production.shape[1], 1

model = tsgm.models.timeGAN.TimeGAN(
    seq_len=seq_len,
    module="gru",
    hidden_dim=24,
    n_features=n_features,
    n_layers=3,
    batch_size=1,
    gamma=1.0,
)
# .compile() sets all optimizers to Adam by default
model.compile()

model.fit(
    data=scaled_actual_production,
    epochs=4,
)

In [ ]:
generated = model.generate(4)